# IMPORTS

In [7]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import logging
import typing
import gc
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
import torch_xla
from torch import nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig, AutoTokenizer, AutoModelForMaskedLM
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

import torch_xla.core.xla_model as xm
#torch_xla._XLAC._xla_get_all_devices()

# if you are running this notebook from 'home/dev/enefit/notebook'. 
os.chdir('..') # else adjust to point to the root of the project.

In [8]:
from sklearn import metrics
from torch import nn, cuda
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from transformers import AutoTokenizer, AutoModel
from kedro_mlflow.io.metrics import MlflowMetricHistoryDataSet
from kedro_mlflow.io.artifacts import MlflowArtifactDataSet
from kedro_mlflow.io.models import MlflowModelSaverDataSet, MlflowModelLoggerDataSet
from typing import Union
import mlflow
import random
import os
import logging
import torch
import pandas as pd
import transformers as hf
import numpy as np

In [9]:
%%capture output

%load_ext kedro.ipython
%reload_kedro

if 'output' in locals() and 'error' in output.stderr:
    output.show()

In [10]:
from src.asrsnlp.pipelines.train_bert import nodes

In [11]:
modelname = catalog.load("params:model_params.name")
llayer = catalog.load("params:model_params.llayer")
dropoutratio = catalog.load("params:model_params.doratio")
max_len = 200
trainparams = catalog.load("params:train_params")

                    INFO     Loading data from 'params:model_params.name' (MemoryDataset)...    ]8;id=387463;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=558514;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:model_params.llayer' (MemoryDataset)...  ]8;id=699623;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=4934;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:model_params.doratio' (MemoryDataset)... ]8;id=264562;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=366451;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

                    INFO     Loading data from 'params:train_params' (MemoryDataset)...         ]8;id=523218;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=839673;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [12]:
model = nodes.ModelClass(modelname, (768,14), dropoutratio)

In [13]:
tokenizer, model = nodes.import_model(modelname, llayer, dropoutratio)

In [14]:
train_feat = catalog.load("train_cleaned_fea")

[11/28/23 21:51:00] INFO     Loading data from 'train_cleaned_fea' (MlflowParquetDataset)...    ]8;id=833796;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=511926;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/kedro/io/data_catalog.py#502\502]8;;\

In [15]:
train_loaded = nodes.loader(train_feat, tokenizer, max_len, trainparams, sample=0.0001)

In [16]:
loss_fn = nodes.get_loss()

In [17]:
optimizer = nodes.get_optimizer(model, 0.01)

In [18]:
device = nodes.get_xla_device()

[11/28/23 21:51:01] WARNING  PJRT is now the default runtime. For more information, see               ]8;id=296489;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py\runtime.py]8;;\:]8;id=534104;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py#35\35]8;;\
                             https://github.com/pytorch/xla/blob/master/docs/pjrt.md                               

                    WARNING  Defaulting to PJRT_DEVICE=CPU                                            ]8;id=629295;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py\runtime.py]8;;\:]8;id=403192;file:///home/ensai/.pyenv/versions/3.10.12/envs/safran-nlp/lib/python3.10/site-packages/torch_xla/runtime.py#46\46]8;;\

In [19]:
torch.cuda.empty_cache()

In [14]:
nodes.train_model(mymodel=model, loss=loss_fn, optim=optimizer, epochs=1, dataloader=train_loaded, device=device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 nodes.train_model(mymodel=model, loss=loss_fn, optim=optimizer, epochs=1, dataloader=tra     │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'model' is not defined

In [20]:
type(train_loaded)

<class 'torch.utils.data.dataloader.DataLoader'>

In [30]:
type(train_loaded).__mro__

(<class 'torch.utils.data.dataloader.DataLoader'>, <class 'typing.Generic'>, <class 'object'>)

In [34]:
type(loss_fn).__mro__


(
    <class 'torch.nn.modules.loss.BCEWithLogitsLoss'>,
    <class 'torch.nn.modules.loss._Loss'>,
    <class 'torch.nn.modules.module.Module'>,
    <class 'object'>
)

In [35]:
type(model).__mro__


(
    <class 'src.asrsnlp.pipelines.train_bert.nodes.ModelClass'>,
    <class 'torch.nn.modules.module.Module'>,
    <class 'object'>
)

In [39]:
"optim" in str(type(optimizer).__mro__)

True

In [43]:
a, b, c = dict(d=2, e=3, f=5).values()

In [44]:
a

2